In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# !pip install opencv-contrib-python==3.4.13.47 --force-reinstall

  Using cached opencv_contrib_python-3.4.13.47-cp37-cp37m-manylinux2014_x86_64.whl (53.7 MB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 3.4.13.47
    Uninstalling opencv-contrib-python-3.4.13.47:
      Successfully uninstalled opencv-contrib-python-3.4.13.47
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2

In [ ]:
# Source: https://drive.google.com/drive/folders/1DzbzBBPj7EFyQhnrtaLLpOIx0N9P5iVH

In [1]:
import numpy as  np
import cv2

In [3]:
# %cd 'drive/MyDrive/Colab Notebooks/yolov4'

/content/drive/MyDrive/Colab Notebooks/yolov4


In [42]:
# load the image to detect, get width, height 

img_to_detect = cv2.imread('testing/3e03b313-66eb-471f-a97a-7b2bf20dcff5.jpg')
img_height = img_to_detect.shape[0]
img_width = img_to_detect.shape[1]

In [43]:
# convert to blob to pass into model
img_blob = cv2.dnn.blobFromImage(img_to_detect, 0.003922, (608, 608), swapRB=True, crop=False)
#recommended by yolo authors, scale factor is 0.003922=1/255, width,height of blob is 320,320
#accepted sizes are 320×320,416×416,608×608. More size means more accuracy but less speed


In [27]:
# class labels 
class_labels = ["red blood cell", "trophozoite", "difficult", "ring", "schizont", "gametocyte", "leukocyte"]

In [28]:
#Declare colors
class_colors = ["0,255,0","0,0,255","255,0,0","255,255,0","0,255,255"]
class_colors = [np.array(every_color.split(",")).astype("int") for every_color in class_colors]
class_colors = np.array(class_colors)
class_colors = np.tile(class_colors,(7,1))


In [29]:
# Loading the malaria custom model 
# input preprocessed blob into model and pass through the model
# obtain the detection predictions by the model using forward() method
yolo_model = cv2.dnn.readNetFromDarknet('darknet/cfg/yolov4-custom.cfg','darknet/backup/yolov4-custom_best_2_66.weights')

In [33]:
# Get all layers from the yolo network
# Loop and find the last layer (output layer) of the yolo network 
yolo_layers = yolo_model.getLayerNames()
yolo_output_layer = [yolo_layers[yolo_layer[0] - 1] for yolo_layer in yolo_model.getUnconnectedOutLayers()]

In [44]:
# input preprocessed blob into model and pass through the model
yolo_model.setInput(img_blob)
# obtain the detection layers by forwarding through till the output layer
obj_detection_layers = yolo_model.forward(yolo_output_layer)

In [45]:
############## NMS Change 1 ###############
# initialization for non-max suppression (NMS)
# declare list for [class id], [box center, width & height[], [confidences]
class_ids_list = []
boxes_list = []
confidences_list = []
############## NMS Change 1 END ###########


# loop over each of the layer outputs
for object_detection_layer in obj_detection_layers:
	# loop over the detections
    for object_detection in object_detection_layer:
        
        # obj_detections[1 to 4] => will have the two center points, box width and box height
        # obj_detections[5] => will have scores for all objects within bounding box
        all_scores = object_detection[5:]
        predicted_class_id = np.argmax(all_scores)
        prediction_confidence = all_scores[predicted_class_id]
    
        # take only predictions with confidence more than 20%
        if prediction_confidence > 0.20:
            #get the predicted label
            predicted_class_label = class_labels[predicted_class_id]
            #obtain the bounding box co-oridnates for actual image from resized image size
            bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
            (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
            start_x_pt = int(box_center_x_pt - (box_width / 2))
            start_y_pt = int(box_center_y_pt - (box_height / 2))
            
            ############## NMS Change 2 ###############
            #save class id, start x, y, width & height, confidences in a list for nms processing
            #make sure to pass confidence as float and width and height as integers
            class_ids_list.append(predicted_class_id)
            confidences_list.append(float(prediction_confidence))
            boxes_list.append([start_x_pt, start_y_pt, int(box_width), int(box_height)])
            ############## NMS Change 2 END ###########
            
############## NMS Change 3 ###############
# Applying the NMS will return only the selected max value ids while suppressing the non maximum (weak) overlapping bounding boxes      
# Non-Maxima Suppression confidence set as 0.5 & max_suppression threhold for NMS as 0.4 (adjust and try for better perfomance)
max_value_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)

# loop through the final set of detections remaining after NMS and draw bounding box and write text
for max_valueid in max_value_ids:
    max_class_id = max_valueid[0]
    box = boxes_list[max_class_id]
    start_x_pt = box[0]
    start_y_pt = box[1]
    box_width = box[2]
    box_height = box[3]
    
    #get the predicted class id and label
    predicted_class_id = class_ids_list[max_class_id]
    predicted_class_label = class_labels[predicted_class_id]
    prediction_confidence = confidences_list[max_class_id]
############## NMS Change 3 END ########### 
           
    end_x_pt = start_x_pt + box_width
    end_y_pt = start_y_pt + box_height
    
    #get a random mask color from the numpy array of colors
    box_color = class_colors[predicted_class_id]
    
    #convert the color numpy array as a list and apply to text and box
    box_color = [int(c) for c in box_color]
    
    # print the prediction in console
    predicted_class_label = "{}: {:.2f}%".format(predicted_class_label, prediction_confidence * 100)
    print("predicted object {}".format(predicted_class_label))
    
    # draw rectangle and text in the image
    cv2.rectangle(img_to_detect, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), box_color, 1)
    cv2.putText(img_to_detect, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1)

predicted object red blood cell: 99.96%
predicted object red blood cell: 99.95%
predicted object red blood cell: 99.95%
predicted object red blood cell: 99.94%
predicted object red blood cell: 99.94%
predicted object red blood cell: 99.94%
predicted object red blood cell: 99.94%
predicted object red blood cell: 99.93%
predicted object red blood cell: 99.93%
predicted object red blood cell: 99.93%
predicted object red blood cell: 99.92%
predicted object red blood cell: 99.92%
predicted object red blood cell: 99.92%
predicted object red blood cell: 99.92%
predicted object red blood cell: 99.92%
predicted object red blood cell: 99.90%
predicted object red blood cell: 99.90%
predicted object red blood cell: 99.89%
predicted object red blood cell: 99.89%
predicted object red blood cell: 99.88%
predicted object red blood cell: 99.88%
predicted object red blood cell: 99.88%
predicted object red blood cell: 99.88%
predicted object red blood cell: 99.87%
predicted object red blood cell: 99.87%


In [46]:
from google.colab.patches import cv2_imshow
cv2_imshow(img_to_detect)

Output hidden; open in https://colab.research.google.com to view.